# Exploration d'un jeu de données avec NILMTK

On travaille sur le jeu de données REDD au format .h5 situé dans 'data/REDD/redd.h5'.
Pour créer ce fichier .h5, pi

La bibliothèque `nilmtk` permet de gérer aisément les différents jeux de données NILM.
Elle utilise un certain nombre de classes visibles sur ce diagramme UML de nilmtk :
https://raw.githubusercontent.com/nilmtk/writing/master/figures/NILMTK_UML.png

Quand on charge un jeu de données, NILMTK créé plusieurs objets `MeterGroup` qui rassemblent pour chaque
bâtiments les différents compteurs disponibles.

In [1]:
%matplotlib inline

In [2]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
import pandas as pd
import nilmtk
from nilmtk import DataSet, MeterGroup

plt.style.use('ggplot')
rcParams['figure.figsize'] = (13, 10)

# load dataset using nilmtk's DataSet object
redd = DataSet('data/REDD/redd.h5')

On peut afficher les metadonnées du dataset chargé :

In [3]:
from nilmtk.utils import print_dict
print_dict(redd.metadata)

Le jeu de données REDD contient 6 bâtiments différents. On peut pour l'instant regarder seulement le premier :

In [4]:
elec = redd.buildings[1].elec
elec

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='REDD', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='REDD', site_meter, appliances=[])
  ElecMeter(instance=5, building=1, dataset='REDD', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=6, building=1, dataset='REDD', appliances=[Appliance(type='dish washer', instance=1)])
  ElecMeter(instance=7, building=1, dataset='REDD', appliances=[Appliance(type='sockets', instance=1)])
  ElecMeter(instance=8, building=1, dataset='REDD', appliances=[Appliance(type='sockets', instance=2)])
  ElecMeter(instance=9, building=1, dataset='REDD', appliances=[Appliance(type='light', instance=1)])
  ElecMeter(instance=11, building=1, dataset='REDD', appliances=[Appliance(type='microwave', instance=1)])
  ElecMeter(instance=12, building=1, dataset='REDD', appliances=[Appliance(type='unknown', instance=1)])
  ElecMeter(instance=13, building=1, dataset='REDD', appliances=[Appliance(type='electric 

In [5]:
for key, building in redd.buildings.items():
    print("Le bâtiment ", key, " contient des sous-compteurs pour ", len(building.elec.appliances), " appareils." )
    for appliance in building.elec.appliances:
        print(appliance.metadata["original_name"])

Le bâtiment  1  contient des sous-compteurs pour  16  appareils.
lighting
electric_heat
kitchen_outlets
stove
kitchen_outlets
washer_dryer
microwave
lighting
refrigerator
bathroom_gfi
oven
lighting
kitchen_outlets
washer_dryer
dishwaser
kitchen_outlets
Le bâtiment  2  contient des sous-compteurs pour  9  appareils.
lighting
kitchen_outlets
disposal
stove
microwave
refrigerator
washer_dryer
dishwaser
kitchen_outlets
Le bâtiment  3  contient des sous-compteurs pour  19  appareils.
outlets_unknown
furnace
kitchen_outlets
microwave
lighting
bathroom_gfi
lighting
outlets_unknown
lighting
washer_dryer
disposal
refrigerator
dishwaser
outlets_unknown
lighting
lighting
kitchen_outlets
smoke_alarms
electronics
Le bâtiment  4  contient des sous-compteurs pour  17  appareils.
lighting
furnace
kitchen_outlets
stove
bathroom_gfi
lighting
miscellaeneous
air_conditioning
lighting
smoke_alarms
kitchen_outlets
washer_dryer
dishwaser
bathroom_gfi
outlets_unknown
lighting
air_conditioning
Le bâtiment  5  

Ce premier bâtiment contient 16 compteurs électriques (`ElecMeter`) différents correspondants aux différents appareils
contenus dans la maison (`elec.appliances`) (frigo, lave vaisselle, chauffage, éclairage ...).
On trouve également deux compteurs `site_meter` pour la consommation globale :

In [5]:
# On peut regarder les données brut de consommation globale :
mains_data = next(elec.mains().load())
mains_data

TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

In [ ]:
elec.mains()

elec.submeters()


## Part de l'énergie totale mesurée par les submeters
Les submeters installés sur les  différents appareils ne mesurent pas forcément toute la consommation
électrique du bâtiment. La fonction `proportion_of_energy_submetered` permet de connaître cette proportion.

On trouve que les sous-compteurs mesurent 76% de la consommation globale du bâtiment.

In [ ]:
elec.proportion_of_energy_submetered()

## Grandeurs mesurées : puissance apparente et puissance réactive


In [6]:
print("mains power type :",elec.mains().available_ac_types('power'))
print("submeters power type :",elec.submeters().available_ac_types('power'))


mains power type : ['apparent']
submeters power type : ['active']


Les compteurs mains mesures la puissance apparente alors que les submeters mesurent la puissance active...


In [7]:
data =  next(elec.load())


TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

In [ ]:
energy_per_meter = elec.submeters().energy_per_meter() # kWh, again
energy_per_meter

# Quelques graphiques avec NILMTK

In [ ]:
elec.plot_power_histogram()

In [ ]:
redd.set_window(start='2011-04-26', end='2011-04-28')
elec.submeters().plot("area");
plt.xlabel("Time");

In [ ]:
elec.plot_when_on(on_power_threshold = 40)

In [ ]:
# Create convenient labels
fraction = elec.submeters().fraction_per_meter().dropna()
labels = elec.get_labels(fraction.index)
plt.figure(figsize=(10,30))
fraction.plot(kind='pie', labels=labels);

# On peut regarder la forme des activations pour un "fridge" par exemple

In [ ]:
redd.set_window(start='2011-04-26', end='2011-05-26')
BUILDING = 1
elec = redd.buildings[BUILDING].elec
fridge = elec['fridge']

activations = fridge.get_activations(border=0)
print("Number of activations : ", len(activations))

for act in activations[:10]:
    plt.figure()
    plt.plot(act)